In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder.enableHiveSupport().master("yarn").getOrCreate()

In [69]:
graph_schema = StructType([
    StructField("to", IntegerType(), False),
    StructField("from", IntegerType(), False)
])

In [23]:
dist_schema = StructType([
    StructField("vertex", IntegerType(), False),
    StructField("distance", IntegerType(), False)
])

In [70]:
from pyspark.sql.functions import col, when

In [112]:
def shortest_path(v_from, v_to, dataset_path):
    edges = spark_session.read.csv(dataset_path, sep="\t", schema=graph_schema)
    edges.cache()
    distances = spark_session.createDataFrame([(v_from, 0)], dist_schema)
    d = 0
    while True:
        candidates = distances.join(edges, distances.vertex == col('from')) \
                              .select(col('to').alias('vertex'), (distances.distance+1).alias('distance'))

        new_distances = distances.join(candidates, on='vertex', how='full_outer') \
                                 .select('vertex', when(
                                                     distances.distance != 0, distances.distance
                                                 ).otherwise(
                                                     candidates.distance
                                                 ).alias('distance')
                                        ).cache()
        count = new_distances.where(new_distances.distance==d+1).count()
        if count > 0:
            d += 1            
            distances = candidates
        else:
            break  
            
        target = (new_distances
                  .where(new_distances.vertex == v_to)
                 ).count()
        if  target > 0:
            break
     
    return d

In [ ]:
length = shortest_path(12, 34, "/data/twitter/twitter_sample.txt")

In [ ]:
spark_session.stop()

In [ ]:
print(length)